In [1]:
from data_reader import Vocabulary, HWDBDatasetHelper, ArchivedHWDBReader

# your path to data
train_path = r'/DATA/asaginbaev/HWDB/HWDBTrain/Images.zip'
test_path = r'/DATA/asaginbaev/HWDB/HWDBTest/Images.zip'
gt_path = './gt.txt'

In [2]:
import cv2
import numpy as np
import matplotlib.pyplot as plt
from tqdm.notebook import tqdm

В общем-то никакого рокет сайнса, в качестве модели взял resnet-18 с модифицированным классификатором и входной сверткой + возвращаю из модели вход классификатора, чтобы использовать ArcFace, изначально прогнал 5 эпох на трейн сплите получилась точность порядка 83%. 5 эпох взял так как эпоха по 6 часов занимала, и задача была посмотреть насколько вообще рабочая модель. Чтобы обучить уже основательно, распаковал всю трейн часть и использовал ImageFolder в качестве датасета. Скорость заметно увеличилась, что можно увидеть ниже. Не придумал как в случае с ImageFolder провернуть разбиение на валидацию и трейн, поэтому решил обучить модель на всей трейн части ради интереса, тем более что +20% данных в трейн сплите- это довольно весомо, обучал 15 эпох, ну и accuracy получилось порядка 93%. Также отказался от простого ресайза к 32 х 32 в пользу пропорционального ресайза картинки до размера, при котором большая сторона равна 64 и паддинга другой стороны до 64, чтобы сохранить исходные пропорции. Чекпоинты с первого и второго из упомянутых выше ранов приложены под именами "checkpoint_solution_1.pth" и "checkpoint_solution_2.pth".

### Data tools

In [64]:
train_reader = ArchivedHWDBReader(train_path)
train_reader.open()
train_helper = HWDBDatasetHelper(train_reader)

In [65]:
train_helper, val_helper = train_helper.train_val_split()

In [66]:
train_helper.size(), val_helper.size()

(2578433, 644609)

In [135]:
import PIL
from itertools import permutations
import zipfile
from typing import Optional, List
from pathlib import Path
import cv2
import numpy as np
from collections import defaultdict, Counter
import torch

from torch.utils.data import Dataset, DataLoader
from torch import nn
from pytorch_metric_learning import losses
import torchvision

In [17]:
def image_transform(img):
    img = np.array(PIL.ImageOps.grayscale(img))
    img_shape = np.array(img.shape)
    new_size = np.array([64, 64])
    intermediate_size = (64 * img_shape / max(img_shape)).astype(int)
    pad_size = new_size - intermediate_size
    side_pad_size = np.array([pad_size // 2, pad_size - pad_size // 2]).T
        
    return np.pad((cv2.resize(img, intermediate_size[::-1]) - 127.5) / 255., side_pad_size, constant_values=0.5)

In [86]:
train_dataset = torchvision.datasets.ImageFolder('/DATA/asaginbaev/HWDB/HWDBTrainUnzipped/CASIA-HWDB_Train/Train',
                                                 transform=image_transform)
#val_dataset = HWDBDataset(val_helper)

### Model & training

In [91]:
class ResNetWrapper(nn.Module):
    def __init__(self, n_classes):
        super(ResNetWrapper, self).__init__()
        self.net = torchvision.models.resnet18(weights=None)
        self.net.conv1 = nn.Conv2d(1, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
        self.net.fc = nn.Identity()
        self.fc = nn.Linear(512, n_classes)
    
    def forward(self, x):
        embs = self.net(x)
        return self.fc(embs), embs

In [92]:
model = ResNetWrapper(train_helper.vocabulary.num_classes())
model.eval();

In [93]:
import os
os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"]="1"
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [94]:
model = model.cuda()

In [122]:
train_loader = DataLoader(train_dataset, batch_size=128, shuffle=True, drop_last=True, num_workers=32)
#val_loader = DataLoader(val_dataset, batch_size=32, shuffle=False)

In [123]:
optim = torch.optim.Adam(model.parameters(), lr=0.001)
loss_fn = torch.nn.CrossEntropyLoss()
loss_metric = losses.ArcFaceLoss(train_helper.vocabulary.num_classes(), 512,)
loss_metric = loss_metric.cuda()
optim_metric = torch.optim.Adam(loss_metric.parameters(), lr=0.001)

In [124]:
from tqdm import tqdm


def run_validation(val_loader: DataLoader, model: nn.Module, n_steps=None):
    model.eval()
    n_good = 0
    n_all = 0
    wrapper = lambda x: x
    if n_steps is None:
        n_steps = len(val_loader)
        wrapper = tqdm
    
    with torch.no_grad():
        for batch, (X, y) in enumerate(wrapper(val_loader)):
            if batch == n_steps:
                break
            logits = model(X.unsqueeze(1).to(torch.float32).cuda())[0]
            classes = torch.argmax(logits, dim=1).cpu().numpy()
            n_good += sum(classes == y.cpu().numpy())
            n_all += len(classes)
    
    return n_good / n_all


def train_epoch(train_loader: DataLoader, model: nn.Module, optim, loss_fn, metric_loss, optim_loss):
    for batch, (X, y) in enumerate(tqdm(train_loader)):
        model.train()
        logits, embs = model(X.unsqueeze(1).to(torch.float32).cuda())
        loss = loss_fn(logits, y.to(torch.long).cuda()) + metric_loss(embs, y.to(torch.long).cuda())
        
        optim.zero_grad()
        loss.backward()
        optim.step()

In [125]:
for epoch in range(15):
    print(f'Epoch {epoch}:')
    train_epoch(train_loader, model, optim, loss_fn, loss_metric, optim_metric)
    torch.save(model.state_dict(), f'baseline_epoch{epoch}.pth')

Epoch 0:


100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████| 25180/25180 [2:56:47<00:00,  2.37it/s]


Epoch 1:


100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 25180/25180 [19:58<00:00, 21.01it/s]


Epoch 2:


...

100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 25180/25180 [15:10<00:00, 27.66it/s]


Epoch 13:


100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 25180/25180 [14:34<00:00, 28.78it/s]


Epoch 14:


100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 25180/25180 [15:23<00:00, 27.28it/s]


### Evaluation

In [126]:
test_path = r'/DATA/asaginbaev/HWDB/HWDBTest/Images.zip'
pred_path = './pred.txt'

test_reader = ArchivedHWDBReader(test_path)
test_reader.open()
test_helper = HWDBDatasetHelper(test_reader, prefix='Test')

In [127]:
test_dataset = HWDBDataset(test_helper)
test_loader = DataLoader(test_dataset, batch_size=16, shuffle=False)

In [128]:
preds = []
model.eval()
with torch.no_grad():
    for X, _ in tqdm(test_loader):
        logits = model(X.unsqueeze(1).to(torch.float32).cuda())[0]
        classes = torch.argmax(logits, dim=1).cpu().numpy()
        preds.extend(classes)

100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 48533/48533 [09:35<00:00, 84.36it/s]


In [129]:
with open(pred_path, 'w') as f_pred:
    for idx, pred in enumerate(preds):
        name = test_helper.namelist[idx]
        cls = train_helper.vocabulary.class_by_index(pred)
        print(name, cls, file=f_pred)

In [131]:
from course_ocr_t2.evaluate import evaluate

In [134]:
evaluate(gt_path, pred_path)

0.9383598425288111

In [133]:
torch.save(model.state_dict(), 'checkpoint_solution_2.pth')